# Inport needed package

- import os, sys # to add the parent directory to the path

In [1]:
import os
import sys
import time

- Using torchvision to create a dataset

In [2]:
# Using torchvision to create a dataset
import cv2
from torchvision import transforms
import torch
from torch.utils.data import random_split, DataLoader
import torchvision

import pandas as pd

- import self library

In [3]:
from train.trainer import ClassifierTrainer as Trainer
import dataset as ds  # type: ignore
import model as md  # type: ignore

c:\Users\nnvuf\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Define classification train process

1. Define place where the model is saved

In [4]:
time_str = time.strftime("%Y%m%d_%H%M%S")

2. Define train function

In [5]:
def train_process(dataset, model):
  # define batch_size
  batch_size = 64

  # init train val test ds
  train_val_size = int(0.8 * len(dataset))
  test_size = len(dataset) - train_val_size
  train_ds, test_ds = random_split(dataset, [train_val_size, test_size])

  # define optimizer using Adam and loss function
  optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
  loss_fn = torch.nn.CrossEntropyLoss()

  trainer = Trainer(model, optimizer, loss_fn, random_seed_value=86)
  print('device: ', trainer.device)
  avg_loss, metric = trainer.cross_validate(train_ds, k=5, epochs=10, batch_size=batch_size)
  print('avg_loss: ', avg_loss)

  # score model
  test_dataloader = DataLoader(test_ds, batch_size=batch_size, shuffle=True)
  model_scored = trainer.score(test_dataloader)
  print(f'model_scored: {model_scored:.4f}, avg_accuracy: {100*(1 - model_scored):.4f}')

  # return model scored, train_avg_lost
  return model_scored, avg_loss

3. execute progress

- define the model

In [6]:
models = [
	torchvision.models.resnet50(weights=torchvision.models.ResNet50_Weights.DEFAULT),
  torchvision.models.densenet121(weights=torchvision.models.DenseNet121_Weights.DEFAULT),
  torchvision.models.vgg16(weights=torchvision.models.VGG16_Weights.DEFAULT)
]

- Define tested datasets

In [7]:
datasets = {
    'gi4e_full': ds.Gi4eDataset(
        './datasets/gi4e',
        transform=transforms.Compose([transforms.ToPILImage(), transforms.Resize((224, 224)), transforms.ToTensor()]),
        is_classification=True),
    'gi4e_raw_eyes': ds.ImageDataset(
        './datasets/gi4e_raw_eyes',
        transform=transforms.Compose([transforms.Resize((224, 224)), transforms.ToTensor()]),
        file_extension='png'),
    'gi4e_detected_eyes': ds.ImageDataset(
        './datasets/gi4e_eyes/20250521_200316',
        transform=transforms.Compose([transforms.Resize((224, 224)), transforms.ToTensor()]),
        file_extension='png'),
}

- Train all defined model on each registered datasets

In [8]:
result_df = pd.DataFrame(columns=['dataset', 'model', 'avg_loss', 'avg_accuracy', 'total_time'])

for name, dataset in datasets.items():
	for model in models:
		print(f'Running {name} dataset with {model.__class__.__name__}')
		# do the train
		start_time = time.time()
		scored, loss = train_process(dataset, model)
		end_time = time.time()
		total_time = end_time - start_time
		print(f'Finished {name} dataset with {model.__class__.__name__}')
		print('----------------------')

		# save the result
		result_df = pd.concat([result_df, pd.DataFrame({
			'model': [model.__class__.__name__],
			'dataset': [name],
			'avg_loss': [loss],
			'avg_accuracy': [scored],
			'total_time': [total_time]
		})], ignore_index=True)

print('Finished all datasets')

# print the result
print(result_df)


Running gi4e_full dataset with ResNet
device:  cuda
Fold 1/5:
Epoch 1/10, Train Loss: 2.2467, Test Loss: 0.7414, Total Time: 00 hours, 00 minutes, 40 seconds
Epoch 2/10, Train Loss: 0.0129, Test Loss: 0.0296, Total Time: 00 hours, 00 minutes, 17 seconds
Epoch 3/10, Train Loss: 0.0015, Test Loss: 0.0058, Total Time: 00 hours, 00 minutes, 17 seconds
Epoch 4/10, Train Loss: 0.0004, Test Loss: 0.0041, Total Time: 00 hours, 00 minutes, 17 seconds
Epoch 5/10, Train Loss: 0.0002, Test Loss: 0.0030, Total Time: 00 hours, 00 minutes, 17 seconds
Epoch 6/10, Train Loss: 0.0001, Test Loss: 0.0025, Total Time: 00 hours, 00 minutes, 17 seconds
Epoch 7/10, Train Loss: 0.0001, Test Loss: 0.0022, Total Time: 00 hours, 00 minutes, 17 seconds
Epoch 8/10, Train Loss: 0.0001, Test Loss: 0.0021, Total Time: 00 hours, 00 minutes, 16 seconds
Epoch 9/10, Train Loss: 0.0001, Test Loss: 0.0020, Total Time: 00 hours, 00 minutes, 17 seconds
Epoch 10/10, Train Loss: 0.0001, Test Loss: 0.0011, Total Time: 00 hours, 

C:\Users\nnvuf\AppData\Local\Temp\ipykernel_15040\454607160.py:15: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  result_df = pd.concat([result_df, pd.DataFrame({


Epoch 1/10, Train Loss: 1.9108, Test Loss: 1.5063, Total Time: 00 hours, 00 minutes, 27 seconds
Epoch 2/10, Train Loss: 0.0348, Test Loss: 0.0855, Total Time: 00 hours, 00 minutes, 20 seconds
Epoch 3/10, Train Loss: 0.0049, Test Loss: 0.0098, Total Time: 00 hours, 00 minutes, 20 seconds
Epoch 4/10, Train Loss: 0.0017, Test Loss: 0.0021, Total Time: 00 hours, 00 minutes, 20 seconds
Epoch 5/10, Train Loss: 0.0009, Test Loss: 0.0013, Total Time: 00 hours, 00 minutes, 19 seconds
Epoch 6/10, Train Loss: 0.0006, Test Loss: 0.0011, Total Time: 00 hours, 00 minutes, 20 seconds
Epoch 7/10, Train Loss: 0.0004, Test Loss: 0.0010, Total Time: 00 hours, 00 minutes, 20 seconds
Epoch 8/10, Train Loss: 0.0003, Test Loss: 0.0010, Total Time: 00 hours, 00 minutes, 20 seconds
Epoch 9/10, Train Loss: 0.0003, Test Loss: 0.0009, Total Time: 00 hours, 00 minutes, 20 seconds
Epoch 10/10, Train Loss: 0.0002, Test Loss: 0.0009, Total Time: 00 hours, 00 minutes, 20 seconds
Fold 1/5, Total Test Loss: 0.0009, Fold

c:\Users\nnvuf\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\nnvuf\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\nnvuf\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, 

Fold 1/5, Total Test Loss: 0.3459, Fold accuracy: 65.4141
Fold 2/5:
Epoch 1/10, Train Loss: 0.5130, Test Loss: 0.1284, Total Time: 00 hours, 00 minutes, 21 seconds
Epoch 2/10, Train Loss: 0.3118, Test Loss: 0.2853, Total Time: 00 hours, 00 minutes, 21 seconds
Epoch 3/10, Train Loss: 0.3201, Test Loss: 0.0998, Total Time: 00 hours, 00 minutes, 22 seconds
Epoch 4/10, Train Loss: 0.2917, Test Loss: 0.1577, Total Time: 00 hours, 00 minutes, 22 seconds
Epoch 5/10, Train Loss: 0.1594, Test Loss: 0.1004, Total Time: 00 hours, 00 minutes, 22 seconds
Epoch 6/10, Train Loss: 0.1700, Test Loss: 0.1236, Total Time: 00 hours, 00 minutes, 21 seconds
Epoch 7/10, Train Loss: 0.1889, Test Loss: 0.0428, Total Time: 00 hours, 00 minutes, 21 seconds
Epoch 8/10, Train Loss: 0.0795, Test Loss: 0.0032, Total Time: 00 hours, 00 minutes, 22 seconds
Epoch 9/10, Train Loss: 0.1511, Test Loss: 0.1153, Total Time: 00 hours, 00 minutes, 21 seconds
Epoch 10/10, Train Loss: 0.1629, Test Loss: 0.0079, Total Time: 00 h

...

Fold 1/5, Total Test Loss: 0.2928, Fold accuracy: 70.7177
Fold 2/5:
Epoch 1/10, Train Loss: 0.0737, Test Loss: 0.7684, Total Time: 00 hours, 00 minutes, 12 seconds
Epoch 2/10, Train Loss: 0.0350, Test Loss: 0.1306, Total Time: 00 hours, 00 minutes, 12 seconds
Epoch 3/10, Train Loss: 0.0111, Test Loss: 0.0126, Total Time: 00 hours, 00 minutes, 12 seconds
Epoch 4/10, Train Loss: 0.0016, Test Loss: 0.0008, Total Time: 00 hours, 00 minutes, 12 seconds
Epoch 5/10, Train Loss: 0.0005, Test Loss: 0.0004, Total Time: 00 hours, 00 minutes, 12 seconds
Epoch 6/10, Train Loss: 0.0003, Test Loss: 0.0003, Total Time: 00 hours, 00 minutes, 12 seconds
Epoch 7/10, Train Loss: 0.0002, Test Loss: 0.0002, Total Time: 00 hours, 00 minutes, 12 seconds
Epoch 8/10, Train Loss: 0.0002, Test Loss: 0.0002, Total Time: 00 hours, 00 minutes, 12 seconds
Epoch 9/10, Train Loss: 0.0002, Test Loss: 0.0002, Total Time: 00 hours, 00 minutes, 12 seconds
Epoch 10/10, Train Loss: 0.0002, Test Loss: 0.0001, Total Time: 00 h

c:\Users\nnvuf\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\nnvuf\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\nnvuf\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capita

Fold 2/5, Total Test Loss: 0.0856, Fold accuracy: 95.5430
Fold 3/5:
Epoch 1/10, Train Loss: 0.0919, Test Loss: 0.0202, Total Time: 00 hours, 00 minutes, 17 seconds
Epoch 2/10, Train Loss: 0.1091, Test Loss: 0.0242, Total Time: 00 hours, 00 minutes, 17 seconds
Epoch 3/10, Train Loss: 0.1111, Test Loss: 0.0253, Total Time: 00 hours, 00 minutes, 17 seconds
Epoch 4/10, Train Loss: 0.0639, Test Loss: 0.0094, Total Time: 00 hours, 00 minutes, 17 seconds
Epoch 5/10, Train Loss: 0.0420, Test Loss: 0.0123, Total Time: 00 hours, 00 minutes, 17 seconds
Epoch 6/10, Train Loss: 0.0563, Test Loss: 0.0072, Total Time: 00 hours, 00 minutes, 17 seconds
Epoch 7/10, Train Loss: 0.0826, Test Loss: 0.0235, Total Time: 00 hours, 00 minutes, 17 seconds
Epoch 8/10, Train Loss: 0.0370, Test Loss: 0.0139, Total Time: 00 hours, 00 minutes, 17 seconds
Epoch 9/10, Train Loss: 0.0756, Test Loss: 0.0360, Total Time: 00 hours, 00 minutes, 17 seconds
Epoch 10/10, Train Loss: 0.0572, Test Loss: 0.0293, Total Time: 00 h

c:\Users\nnvuf\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\nnvuf\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\nnvuf\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, mo

Fold 5/5, Total Test Loss: 0.2926, Fold accuracy: 93.6570
avg_loss:  0.058516472305713677
model_scored: 0.2665, avg_accuracy: 73.3485
Finished gi4e_detected_eyes dataset with VGG
----------------------
Finished all datasets
              dataset     model  avg_loss  avg_accuracy   total_time
0           gi4e_full    ResNet  0.000215      0.000002   962.041694
1           gi4e_full  DenseNet  0.000198      0.000030  1046.056977
2           gi4e_full       VGG  0.091103      0.083772  1207.610080
3       gi4e_raw_eyes    ResNet  0.147899      0.135539   540.670171
4       gi4e_raw_eyes  DenseNet  0.006350      0.031950   590.097127
5       gi4e_raw_eyes       VGG  0.535764      0.562497   932.827783
6  gi4e_detected_eyes    ResNet  0.023560      0.011821   560.538603
7  gi4e_detected_eyes  DenseNet  0.058603      0.031782   630.374697
8  gi4e_detected_eyes       VGG  0.058516      0.266515   962.397992


4. print the result

In [9]:
# swap the first two columns
result_df = result_df[['dataset', 'model', 'avg_loss', 'avg_accuracy', 'total_time']]
# scale the avg_accuracy to 0-100
result_df['avg_accuracy'] = 100 * (1 - result_df['avg_accuracy'])
# display the total time in the format HH:MM:SS
result_df['total_time'] = pd.to_timedelta(result_df['total_time'], unit='s')

# save the result to csv
result_df.to_csv(f'results_{time_str}.csv', index=False)
# print the result
print(result_df)


              dataset     model  avg_loss  avg_accuracy  \
0           gi4e_full    ResNet  0.000215     99.999779   
1           gi4e_full  DenseNet  0.000198     99.996984   
2           gi4e_full       VGG  0.091103     91.622755   
3       gi4e_raw_eyes    ResNet  0.147899     86.446137   
4       gi4e_raw_eyes  DenseNet  0.006350     96.805048   
5       gi4e_raw_eyes       VGG  0.535764     43.750337   
6  gi4e_detected_eyes    ResNet  0.023560     98.817859   
7  gi4e_detected_eyes  DenseNet  0.058603     96.821776   
8  gi4e_detected_eyes       VGG  0.058516     73.348478   

                 total_time  
0 0 days 00:16:02.041694403  
1 0 days 00:17:26.056977272  
2 0 days 00:20:07.610080004  
3 0 days 00:09:00.670171022  
4 0 days 00:09:50.097127438  
5 0 days 00:15:32.827783108  
6 0 days 00:09:20.538602591  
7 0 days 00:10:30.374696732  
8 0 days 00:16:02.397992134  


In [10]:
print(result_df)

              dataset     model  avg_loss  avg_accuracy  \
0           gi4e_full    ResNet  0.000215     99.999779   
1           gi4e_full  DenseNet  0.000198     99.996984   
2           gi4e_full       VGG  0.091103     91.622755   
3       gi4e_raw_eyes    ResNet  0.147899     86.446137   
4       gi4e_raw_eyes  DenseNet  0.006350     96.805048   
5       gi4e_raw_eyes       VGG  0.535764     43.750337   
6  gi4e_detected_eyes    ResNet  0.023560     98.817859   
7  gi4e_detected_eyes  DenseNet  0.058603     96.821776   
8  gi4e_detected_eyes       VGG  0.058516     73.348478   

                 total_time  
0 0 days 00:16:02.041694403  
1 0 days 00:17:26.056977272  
2 0 days 00:20:07.610080004  
3 0 days 00:09:00.670171022  
4 0 days 00:09:50.097127438  
5 0 days 00:15:32.827783108  
6 0 days 00:09:20.538602591  
7 0 days 00:10:30.374696732  
8 0 days 00:16:02.397992134  


# Embedded Classification

- Define the model

In [11]:
embedded_models = [md.FeatureExtractor(model) for model in models]
classifier_models = [md.Classifier(model, 103) for model in embedded_models]


- Load the trained weight to the model

In [12]:
# ignore

- Transform the dataset to the embedded data

In [13]:
classifier_df = pd.DataFrame(columns=['key', 'dataset', 'model'])

for name, dataset in datasets.items():
	for model in embedded_models:
		print(f'Getting features for {name} dataset with {model._get_name()}')

		# get the classifier model from the model
		classifier_model = next((m for m in classifier_models if m.backbone._get_name() == model._get_name()), None)
		if classifier_model is None:
			print(f'Classifier model not found for {model._get_name()}')
			continue

		model_dataset = ds.EmbeddedDataset(dataset, model)

		classifier_df = pd.concat([classifier_df, pd.DataFrame({
				'key': [f'{name}_{model._get_name()}'],
				'model': [classifier_model],
				'dataset': [model_dataset]
		})], ignore_index=True)

		print(f'Finished getting features for {name} dataset with {model._get_name()}')


Getting features for gi4e_full dataset with FeatureExtractor(ResNet)


<All keys matched successfully>


RuntimeError: Input type (torch.FloatTensor) and weight type (torch.cuda.FloatTensor) should be the same or input should be a MKLDNN tensor and weight is a dense tensor

- Train all defined model on each registered datasets

In [ ]:
result_df = pd.DataFrame(columns=['dataset', 'model', 'avg_loss', 'avg_accuracy', 'total_time'])

for index, row in classifier_df.iterrows():
	dataset = row['dataset']
	model = row['model']
	key = row['key']

	print(f'Running {key} dataset with {model._get_name()}')
	# do the train
	start_time = time.time()
	scored, loss = train_process(dataset, model)
	end_time = time.time()
	total_time = end_time - start_time
	print(f'Finished {key} dataset with {model._get_name()}')
	print('----------------------')

	# save the result
	result_df = pd.concat([result_df, pd.DataFrame({
			'model': [model._get_name()],
			'dataset': [dataset.__class__.__name__],
			'avg_loss': [loss],
			'avg_accuracy': [scored],
			'total_time': [total_time]
	})], ignore_index=True)
print('Finished training all models')


- Print the result

In [ ]:
# swap the first two columns
result_df = result_df[['dataset', 'model', 'avg_loss', 'avg_accuracy', 'total_time']]
# scale the avg_accuracy to 0-100
result_df['avg_accuracy'] = 100 * (1 - result_df['avg_accuracy'])
# display the total time in the format HH:MM:SS
result_df['total_time'] = pd.to_timedelta(result_df['total_time'], unit='s')

# save the result to csv
result_df.to_csv(f'results_{time_str}.csv', index=False)
# print the result
print(result_df)


In [ ]:
models = [
	torchvision.models.resnet50(weights=torchvision.models.ResNet50_Weights.DEFAULT),
  torchvision.models.densenet121(weights=torchvision.models.DenseNet121_Weights.DEFAULT),
  torchvision.models.vgg16(weights=torchvision.models.VGG16_Weights.DEFAULT)
]

for model in models:
  # explore the model
	print(f'Model: {model.__class__.__name__}')
	
	# explain the model
	print(f'Number of layers: {len(list(model.children()))}')
	print(model)